In [1]:
from collections import Counter
import urllib.request
import pandas as pd
import numpy as np
import tensorflow as tf

import nltk
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# spam classification data loading
urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin-1')
data[:5]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
texts = list(data['v2'])
labels = list(data['v1'])

print(texts[:5])
print(labels[:5])

print(Counter(labels))

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'U dun say so early hor... U c already then say...', "Nah I don't think he goes to usf, he lives around here though"]
['ham', 'ham', 'spam', 'ham', 'ham']
Counter({'ham': 4825, 'spam': 747})


In [4]:
def tokenize(document):
  words = []
  sentences = sent_tokenize(document) # sentence tokenizing

  for sentence in sentences:
    words.extend(WordPunctTokenizer().tokenize(sentence)) # word tokenizing

  return [word.lower() for word in words] # case normalization

def make_vocab(documents):
  word2index = {'<unk>':0, '<pad>':1}
  for document in documents:
    tokens = tokenize(document)
    for token in tokens:
      if token not in word2index.keys():
        word2index[token] = len(word2index)

  return word2index

def get_vector(sentence, vocab):
  tokens = tokenize(sentence)
  vector = [vocab[token] if token in vocab.keys() else vocab['<unk>'] for token in tokens]

  while len(vector) < 256:
    vector.append(vocab['<pad>'])
  
  return vector[:256]

In [6]:
vocab = make_vocab(texts)
print(vocab)
print(texts[0])
print(tokenize(texts[0]))
print(get_vector(texts[0], vocab))

{'<unk>': 0, '<pad>': 1, 'go': 2, 'until': 3, 'jurong': 4, 'point': 5, ',': 6, 'crazy': 7, '..': 8, 'available': 9, 'only': 10, 'in': 11, 'bugis': 12, 'n': 13, 'great': 14, 'world': 15, 'la': 16, 'e': 17, 'buffet': 18, '...': 19, 'cine': 20, 'there': 21, 'got': 22, 'amore': 23, 'wat': 24, 'ok': 25, 'lar': 26, 'joking': 27, 'wif': 28, 'u': 29, 'oni': 30, 'free': 31, 'entry': 32, '2': 33, 'a': 34, 'wkly': 35, 'comp': 36, 'to': 37, 'win': 38, 'fa': 39, 'cup': 40, 'final': 41, 'tkts': 42, '21st': 43, 'may': 44, '2005': 45, '.': 46, 'text': 47, '87121': 48, 'receive': 49, 'question': 50, '(': 51, 'std': 52, 'txt': 53, 'rate': 54, ')': 55, 't': 56, '&': 57, 'c': 58, "'": 59, 's': 60, 'apply': 61, '08452810075over18': 62, 'dun': 63, 'say': 64, 'so': 65, 'early': 66, 'hor': 67, 'already': 68, 'then': 69, 'nah': 70, 'i': 71, 'don': 72, 'think': 73, 'he': 74, 'goes': 75, 'usf': 76, 'lives': 77, 'around': 78, 'here': 79, 'though': 80, 'freemsg': 81, 'hey': 82, 'darling': 83, 'it': 84, 'been': 85,

In [8]:
x = [get_vector(text, vocab) for text in texts]
x_train, x_test = np.array(x[:5000]), np.array(x[5000:])
y = [0 if label == 'spam' else 1 for label in labels]
y_train, y_test = np.array(y[:5000]), np.array(y[5000:])
print(x_train.shape)

(5000, 256)


In [10]:
input_layer = tf.keras.layers.Input(shape=(x_train.shape[1]))
embedding_table = tf.keras.layers.Embedding(len(vocab), 100)
embedded_layer = tf.keras.layers.Dropout(0.5)(embedding_table(input_layer))
lstm_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(embedded_layer)
attention = tf.keras.layers.Attention()([lstm_layer, lstm_layer, lstm_layer])
flatten = tf.keras.layers.Flatten()(attention)
dense1_layer = tf.keras.layers.Dense(64, activation = 'relu')(flatten)
dense2_layer = tf.keras.layers.Dense(2, activation = 'softmax')(dense1_layer)

model = tf.keras.Model(inputs=input_layer, outputs=dense2_layer)

In [11]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 256, 100)     892400      input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 256, 100)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 256, 256)     234496      dropout_1[0][0]                  
_______________________________________________________________________________________

In [12]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=2)

Epoch 1/2
 12/157 [=>............................] - ETA: 1:44 - loss: 0.5448 - accuracy: 0.7292

In [ ]:
model.evaluate(x_test, y_test, verbose=2)